In [3]:
import spacy, coreferee
from spacy.matcher import Matcher
import pandas as pd


In [4]:
nlp = spacy.load("en_core_web_lg")
nlp.max_length = 3200000
nlp.add_pipe('coreferee')


In [21]:
import re
def import_file(file_name):
  text = ""


  with open(file_name, encoding="utf8") as f:
    try:
      text = f.readlines()
    except Exception as e:
      print(e)



  string_text = ""
  for line in text:
    string_text += line + "\n"
  string_text = re.sub('[\n]+', '\n', string_text)
  return string_text


In [7]:
string_text[:1000]

'A beginning is the time for taking the most delicate care that the balances are correct. This every sister of the Bene Gesserit knows. To begin your study of the life of Muad‘Dib, then, take care that you first place him in his time: born in the 57th year of the Padishah Emperor, Shaddam IV. And take the most special care that you locate Muad’Dib in his place: the planet Arrakis. Do not be deceived by the fact that he was bom on Caladan and lived his first fifteen years there. Arrakis, the planet known as Dune, is forever his place.\n-from “Manual of Muad’Dib” by the Princess Irulan\nIN THE week before their departure to Arrakis, when all the final scurrying about had reached a nearly unbearable frenzy, an old crone came to visit the mother of the boy, Paul.\nIt was a warm night at Castle Caladan, and the ancient pile of stone that had served the Atreides family as home for twenty-six generations bore that cooled-sweat feeling it acquired before a change in the weather.\nThe old woman

In [8]:
doc = nlp(string_text)


In [9]:
doc.to_disk("./DUNE_spacy.json")

In [1]:
import pickle

pickle.dump(doc, open("./Dumps/Dune/DUNE_spacy.pkl", "wb"))

NameError: name 'doc' is not defined

In [2]:
import pickle

In [5]:
newDoc = pickle.load(open("./Dumps/Dune/DUNE_spacy.pkl", "rb"))

In [ ]:
newDoc._.coref_chains.print()


In [14]:
coreference_resolved = ""
for document in doc:
  res = doc._.coref_chains.resolve(document)
  coreference_resolved += res[0].text +" " if res else document.text + " "


In [116]:
with open("LOTR_COREF.txt", "w", encoding="utf8") as f:
  f.write(coreference_resolved)

In [39]:
test = coreference_resolved[:500].split(".")

In [ ]:
test[:200]

In [22]:
textString = import_file("./LOTR_COREF.txt")

In [24]:
textString[:100]

'\n Chapter 1 \n A LONG - EXPECTED PARTY \n When Mr. Bilbo Baggins of Bag End announced that Baggins wou'

In [23]:
doc = nlp(textString)


In [25]:
import pickle

pickle.dump(doc, open("./Dumps/LOTR/LOTR_spacy.pkl", "wb"))


In [28]:
newDoc = doc

In [28]:
doc.to_disk("./LOTR1_spacy.json")


In [11]:
for i in range(10):
  print(doc[i].text, "-->",doc[i].dep_,"-->", doc[i].pos_)

A --> det --> DET
beginning --> nsubj --> NOUN
is --> ROOT --> AUX
the --> det --> DET
time --> attr --> NOUN
for --> prep --> ADP
taking --> pcomp --> VERB
the --> det --> DET
most --> advmod --> ADV
delicate --> amod --> ADJ


In [ ]:
i=0
for token in doc:
  if token.pos_ == 'PRON' and token._.in_coref:
    for cluster in token._.coref_clusters:

      print(token.text, "-->",cluster.main.text)
    if i == 10:
      break
    else:
      i+=1 

In [4]:
pattern = [
    [ {"POS": "VERB"}, {"POS": "PART", "OP": "*"}, {"POST": "ADV", "OP": "*"},{"POS": {"IN": ["NOUN", "PRON", "PROPN"]}}],
    [ {"POS": "VERB"}, {"POST": "ADJ",
                                                                   "OP": "*"}, {"POST": "ADP",
                                                                                "OP": "*"},{"POST": "ADV", "OP": "*"}, ],
]
pattern1 =  [ {"POS": "VERB"}, {"POS": "PART", "OP": "*"}, {"POST": "ADV", "OP": "*"}]


In [26]:
pattern2 = [
    [{"POS": {"IN": ["PROPN"]}, "OP":"*"},{"POS": {"IN": ["PROPN"]}, "OP":"+"}, {"POS": "VERB"}, {"POS": "PART", "OP": "*"}, {"POS": "ADV", "OP": "*"}, {"POS": {"IN": ["PROPN"]}, "OP": "+"}],
    #Second pattern
    [{"POS": {"IN": ["PROPN"]}, "OP":"*"},{"POS": {"IN": ["PROPN"]}, "OP":"+"}, {"POS": "VERB"},
     {"POS": {"IN": ["PART", "ADV", "CONJ"]}, "OP": "*"},  {"POS": {"IN": [ "ADV"]}, "OP": "+"},{"POS": {"IN": ["PROPN", "NOUN"]}, "OP":"+"}],
    #third pattern
    [{"POS": {"IN": ["PROPN"]}, "OP":"*"},{"POS": {"IN": ["PROPN"]}, "OP":"+"}, {"POS": "VERB"},
     {"POS": {"IN": ["PART", "ADV", "CONJ"]}, "OP": "*"}, {"POS": {"IN": ["ADV"]}, "OP": "+"}, {"POS": {"IN": ["PROPN", ]}, "OP":"+"}, {"POS": {"IN": ["PART", "ADV", "CONJ", "ADJ", "VERB"]}, "OP": "*"},  {"POS": {"IN": ["PROPN", "NOUN"]}, "OP":"+"}]
    ]


Try matching dependency tree

In [30]:
matcher = Matcher(nlp.vocab)
matcher.add("initial",pattern2)


In [31]:
matches = matcher(newDoc)

In [ ]:
matches[:30]

In [32]:
len(matches)

328

In [33]:
def longest_string_array(array):
    longest_string = ""
    for string in array:
        if len(string) > len(longest_string):
            longest_string = string
    return longest_string

In [34]:
filteredarray = []
for match in matches:
  start = match[1]
  end = match[2]
  tempArray = []
  for new_match in matches:
    if new_match[1] == start or new_match[2] == end:
      tempArray.append(newDoc[new_match[1]:new_match[2]])
  if len(tempArray) > 1:
    sentence = longest_string_array(tempArray)

    if sentence.text != "" and sentence not in filteredarray:
      filteredarray.append(sentence)
  else:
    filteredarray.append(newDoc[start:end])


In [35]:
len(filteredarray)

305

In [36]:
filteredarray[:20]

[Bilbo adopted Frodo,
 Otho disliked Bilbo,
 Bilbo left Bilbo,
 Old Odo Proudfoot removed Bilbo,
 Baggins slipped Baggins,
 Bilbo drew Bilbo,
 Bilbo shrugged Bilbo,
 Bilbo chose Bilbo,
 Bilbo snapped Bilbo,
 Frodo escorted Lobelia,
 Frodo saw Gandalf,
 Frodo shrugged Frodo,
 Bilbo found Bilbo,
 Sam waved Sam,
 Sam believed Sam,
 Ted drained Ted,
 Gandalf paid Frodo,
 Frodo welcomed Sam,
 Sam pressed Gandalf,
 Bilbo found Bilbo]

In [76]:
test = nlp(filteredarray[3].text)

In [82]:
print(test[3].pos_)


VERB


In [37]:
triples = pd.DataFrame(columns=["ENTITY1", "RELATION", "ENTITY2"])
for relation in filteredarray:
  processed_relation = nlp(relation.text)
  i = 0
  entities = []
  relation = ""
  while i < len(processed_relation):
    token = processed_relation[i]
    entity_name = ""
    print(token.text)
    if token.pos_ == "PROPN" or token.pos_ == "NOUN":
      entity_name += token.text
      j = i+1
      while j <len(processed_relation) and (processed_relation[j].pos_ == "PROPN" or processed_relation[j].pos_ == "NOUN"):
        entity_name += " " + processed_relation[j].text
        j += 1
      i = j-1
      if len(entities) <2 :
        entities.append(entity_name)
      else:
        entities[1] += " "+ entity_name
    else:
      relation +=" "+ token.text
    
    i+=1
  print(entities)
  print(relation)
  if len(entities) == 2:
    splitted_second = entities[1].split(" ")
    if len(splitted_second) > 1 and entities[0] == splitted_second[0]:
      triples.loc[len(triples)] = [entities[0], relation,
                                   splitted_second[1]]
    else:
      triples.loc[len(triples)] = [entities[0], relation, entities[1]]
  

Bilbo
adopted
Frodo
['Bilbo', 'Frodo']
 adopted
Otho
disliked
Bilbo
['Otho', 'Bilbo']
 disliked
Bilbo
left
Bilbo
['Bilbo', 'Bilbo']
 left
Old
removed
Bilbo
['Old Odo Proudfoot', 'Bilbo']
 removed
Baggins
slipped
Baggins
['Baggins', 'Baggins']
 slipped
Bilbo
drew
Bilbo
['Bilbo', 'Bilbo']
 drew
Bilbo
shrugged
Bilbo
['Bilbo', 'Bilbo']
 shrugged
Bilbo
chose
Bilbo
['Bilbo', 'Bilbo']
 chose
Bilbo
snapped
Bilbo
['Bilbo', 'Bilbo']
 snapped
Frodo
escorted
Lobelia
['Frodo', 'Lobelia']
 escorted
Frodo
saw
Gandalf
['Frodo', 'Gandalf']
 saw
Frodo
shrugged
Frodo
['Frodo', 'Frodo']
 shrugged
Bilbo
found
Bilbo
['Bilbo', 'Bilbo']
 found
Sam
waved
Sam
['Sam', 'Sam']
 waved
Sam
believed
Sam
['Sam', 'Sam']
 believed
Ted
drained
Ted
['Ted', 'Ted']
 drained
Gandalf
paid
Frodo
['Gandalf', 'Frodo']
 paid
Frodo
welcomed
Sam
['Frodo', 'Sam']
 welcomed
Sam
pressed
Gandalf
['Sam', 'Gandalf']
 pressed
Bilbo
found
Bilbo
['Bilbo', 'Bilbo']
 found
Nine
became
Ringwraiths
['Ringwraiths']
 Nine became
Orcs
saw
Isildur


In [26]:
len(triples)

733

In [38]:
triples.to_csv("LOTR2_TRIPLES.csv")

In [39]:
triples.head(20)

,ENTITY1,RELATION,ENTITY2
0,Bilbo,adopted,Frodo
1,Otho,disliked,Bilbo
2,Bilbo,left,Bilbo
3,Old Odo Proudfoot,removed,Bilbo
4,Baggins,slipped,Baggins
5,Bilbo,drew,Bilbo
6,Bilbo,shrugged,Bilbo
7,Bilbo,chose,Bilbo
8,Bilbo,snapped,Bilbo
9,Frodo,escorted,Lobelia
